### What is this?
In #7 we tried to use Seq2Seql but we couldn't figure it out.
Here we're going to use [working example](https://github.com/nicolas-ivanov/debug_seq2seq) step by step, so that we should be able to know what's going on.

### Data
We gonna use very small data set for faster iterations. Once we're done with whole process, we will increase data size.

### word2vec
Here we convert our vocabrary to vector, so that we can have smaller dimension and similarities between words.

In [2]:
from gensim.models import Word2Vec

window = 5              # window is the maximum distance between the current and predicted word within a sentence.
min_count = 1           # ignore all words with total frequency lower than this.
max_vocab_size = 20000  # limit RAM during vocabulary building; if there are more unique words than this, then prune the infrequent ones. Every 10 million word types need about 1GB of RAM. Set to None for no limit (default).
size = 256              # size is the dimensionality of the feature vectors.
workers = 5             # use this many worker threads to train the model (=faster training with multicore machines

model = Word2Vec(window=window,
                 min_count=min_count,
                 max_vocab_size=max_vocab_size,
                 size=size,
                 workers=workers)
print(model)

model.build_vocab(tokenized_lines_for_voc)
model.train(tokenized_lines_for_train)



Word2Vec(vocab=0, size=256, alpha=0.025)
